In [47]:
import soundfile as sf
import librosa
import numpy as np

In [3]:
#working directory
import os
workDir = os.getcwd()

In [55]:
#grab first file
firstSoundFile = sf.SoundFile('C:/Users/Blake/Documents/Projects/Speech-Emotion Recogntion/Actors/Actor_01/03-01-01-01-01-01-01.wav')

In [56]:
#read file
data = firstSoundFile.read()
sr = firstSoundFile.samplerate

In [57]:
#librosa stats
stft = np.abs(librosa.stft(data))
mfccs = np.mean(librosa.feature.mfcc(y=data,sr=sr, n_mfcc=40).T, axis=0)
chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T,axis=0)

In [62]:
#close to prevent overwriting
firstSoundFile.close()